# Treinamento do Modelo

### 1. Importação das Bibliotecas

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, accuracy_score

### 2. Configuração do Dispositivo

Verifica se há GPU disponível e informa qual dispositivo será usado.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

### 3. Transformações das Imagens

Cada imagem do EMNIST tem tamanho 28x28.
Transformamos em tensor e achatamos em um vetor de 784 valores (28×28).

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.view(-1))
])

### 4. Carregamento do Dataset EMNIST Letters

A base EMNIST Letters contém letras manuscritas.
Aqui carregamos os dados de treino e teste, já aplicando as transformações.

In [ ]:
train_dataset = datasets.EMNIST(root='data', split='letters', train=True, download=True, transform=transform)
test_dataset = datasets.EMNIST(root='data', split='letters', train=False, download=True, transform=transform)

### 5. Conversão para Problema Binário (A vs Não-A)

O dataset original possui 26 classes (A–Z).
Transformamos em um problema binário:

   * 1 → letra A

   * 0 → qualquer outra letra

In [ ]:
train_targets = torch.where(train_dataset.targets == 1, 1, 0)
test_targets = torch.where(test_dataset.targets == 1, 1, 0)
train_dataset.targets = train_targets
test_dataset.targets = test_targets

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

### 6. Definição do Modelo Perceptron

Um Perceptron simples com:

   * Entrada: 784 neurônios (28×28 pixels)

   * Saída: 1 neurônio (resultado binário)

In [ ]:
class Perceptron(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(28*28, 1)
        
    def forward(self, x):
        return self.fc(x)

model = Perceptron().to(device)

### 7. Correção de Desbalanceamento

Como a letra “A” aparece menos vezes, aplicamos peso maior para a classe positiva (A)
usando pos_weight na função de perda BCEWithLogitsLoss.

In [ ]:
pos_weight = torch.tensor([4.0]).to(device) 
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = optim.SGD(model.parameters(), lr=0.1)

### 8. Treinamento do Modelo

Treinamos por 30 épocas, calculando a perda média em cada uma.

In [ ]:
print("Iniciando o treinamento (v5.0 - Mais Calmo)...")
for epoch in range(30):
    model.train()
    total_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.float().to(device)
        
        optimizer.zero_grad()
        outputs = model(images).squeeze()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    print(f"Época {epoch+1}/30 - Loss: {total_loss/len(train_loader):.4f}")

print("Treinamento concluído.")

### 9. Avaliação no Conjunto de Teste

Usamos o modelo para prever as classes e calcular:

   * Acurácia

   * Relatório de classificação (precision, recall, f1-score)

In [ ]:
model.eval()
y_true, y_pred = [], []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.float().to(device)
        outputs = torch.sigmoid(model(images)).squeeze()
        preds = (outputs > 0.5).float()
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

acc = accuracy_score(y_true, y_pred)
print(f"\nPrecisão final no set de teste (v5.0): {acc*100:.2f}%")
print(classification_report(y_true, y_pred, target_names=["Não-A", "A"]))

### 10. Salvando o Modelo Treinado

O modelo é salvo em arquivo .pth para uso posterior.

In [ ]:
torch.save(model.state_dict(), 'perceptron_A_v5.pth')
print("\nModelo treinado (v5.0) salvo como 'perceptron_A_v5.pth'")

# Perceptron Funcionando

### 1. Importações das Bibliotecas

In [ ]:
import sys

# Bibliotecas de IA
import torch
import torch.nn as nn
import torch.optim as optim

# Bibliotecas de Imagem
from torchvision import transforms
from PIL import Image
from io import BytesIO 

# Imports de QWidget e Layouts (Ignorados na explicação do modelo)
from PyQt5.QtWidgets import (QApplication, QWidget, QPushButton, QVBoxLayout, QHBoxLayout, 
                             QFileDialog, QLabel, QMessageBox, QDialog, QDialogButtonBox,
                             QStackedWidget)
# Imports de QGeral e QGraficos (Ignorados na explicação do modelo)
from PyQt5.QtCore import Qt, QPoint, QBuffer, QIODevice
from PyQt5.QtGui import QPixmap, QIcon, QPainter, QPen, QImage

### 2. Arquitetura do Modelo (Perceptron)

Antes de qualquer interface aparecer, o script prepara a IA. Ele precisa recriar a arquitetura do Perceptron e carregar o "cérebro" treinado (.pth).

* **Define a Arquitetura:** O script define exatamente a mesma classe Perceptron que existe no script de treino.



In [ ]:
# Arquitetura do Modelo
class Perceptron(nn.Module):
    def __init__(self):
        super().__init__()
        # Entrada: 28*28 = 784 pixels. Saída: 1 logit.
        self.fc = nn.Linear(28*28, 1)

### 3. Carregamento do Modelo

*  **`device`**: Define onde o PyTorch executará os cálculos.
*  **`model.load_state_dict(...)`**: Carrega os "conhecimento" do arquivo `.pth` para a arquitetura do modelo.
*  **`model.eval()`**: Coloca o modelo em "modo de avaliação". Ele informa ao PyTorch que não precisa calcular gradientes para inferência, para deixar o processo mais rápido.

In [ ]:
# Carregar o Modelo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Perceptron().to(device)

MODEL_FILE = 'perceptron_A_v5.pth'
try:
    # Carrega os pesos salvos no disco
    model.load_state_dict(torch.load(MODEL_FILE))
except FileNotFoundError:
    print(f"ERRO: Arquivo '{MODEL_FILE}' não encontrado.")
    print("Por favor, rode o script 'treinar_modelo.py' (v5.0) primeiro.")
    sys.exit()

# Coloca em modo de inferência
model.eval()

### 4. Transformações de Pré-processamento da Imagem

Prepara uma serie de processamento (`data_transform`) que formata qualquer imagem para o padrão 28x28, converte para escala de cinza, inverte as cores (letra branca/fundo preto) e "achatada" em um vetor.

In [ ]:
# Transformações da Imagem
data_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1), # Converte para cinza
    transforms.Resize((28, 28)),                 # Redimensiona para 28x28
    transforms.ToTensor(),                       # Converte para Tensor
    transforms.Lambda(lambda x: 1.0 - x),        # Inverte as cores
    transforms.Lambda(lambda x: x.view(-1))      # Achata o tensor
])

### 5. Estrutura da Interface

*  HomePage (Linha 160): A tela inicial. Permite ao usuário "Procurar Imagem...". Quando uma imagem é carregada, o mesmo botão muda para "Testar Imagem".

In [ ]:
# Página Inicial
class HomePage(QWidget):
    # ...
    def on_action_click(self):
        if self.btn_action.text() == 'Procurar Imagem...':
            # Carrega a imagem
        else:
            # Testa a imagem
            self.parent_app.run_test(self.image_tensor)

* DrawingPage (Linha 101): A tela de desenho. Ela fornece um "canvas" branco onde o usuário pode desenhar com o mouse. Ao clicar em "Testar Desenho", ela converte o desenho em uma imagem e a envia para o teste.


In [ ]:
#  Página de Desenho
class DrawingPage(QWidget):
    
    def mouseMoveEvent(self, event):
        # Lógica para desenhar na tela (pixmap)

    def test_drawing(self):
        # Converte o pixmap para imagem e chama o run_test
        self.parent_app.run_test(image_tensor)

### 6. Cérebro da Interação

As duas funções mais importantes na classe App, que definem o fluxo principal.

* run_test (Linha 300): Esta é a função central de teste. Ela é chamada tanto pela HomePage quanto pela DrawingPage.

    * Recebe a imagem processada
    * Passa a imagem pelo modelo `(model(image_tensor_to_test))` para obter uma previsão.
    * Calcula a probabilidade `(torch.sigmoid)`.
    * Cria e exibe a janela de diálogo `(FeedbackDialog)` mostrando o resultado.
    * Se o usuário corrigiu o modelo ("Ele Errou!"), ele chama a próxima função: `aprender_com_feedback`.

In [ ]:
# FUNÇÃO CENTRAL
def run_test(self, image_tensor_to_test):
    # ...
    probabilidade = torch.sigmoid(output_raw).item()
    # ...
    dialog = FeedbackDialog('Resultado', msg, self)
    dialog.exec_() 

    # LÓGICA DE APRENDIZADO
    if dialog.feedback == 'errado':
        # ...
        self.aprender_com_feedback(image_tensor_to_test, label_correta, show_thank_you_message=True)

* `aprender_com_feedback` (Linha 352): Esta é a função que realiza o Aprendizado Contínuo.

    * Coloca o modelo em modo de treino `(model.train()).`
    * Realiza o passo de aprendizado/backpropagation.
    * Coloca o modelo de volta em modo de avaliação `(model.eval())`.

In [ ]:
# Função de Aprendizado
def aprender_com_feedback(self, image_tensor, label_correta, show_thank_you_message):
    model.train() # Coloca em modo de treino
    optimizer.zero_grad()
    # ...
    loss = criterion(output, label_tensor)
    loss.backward()
    optimizer.step()
    model.eval() # Volta ao modo de avaliação

    torch.save(model.state_dict(), MODEL_FILE) # Salva o modelo!